## Object Detection

![Image Algorithms](https://cdn-images-1.medium.com/max/840/1*Hz6t-tokG1niaUfmcysusw.jpeg)

## Naive Approach to Object Detection

* Break image into tiles
* Run CNN image classified through each time
* Select tiles with activation above the threshold

In [ ]:
%pip install opencv-python
import cv2
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os

Let's read sample image to play with and pad it to square dimension:

In [ ]:
# OpenCV와 Matplotlib를 사용하여 이미지를 불러오고, 색상 변환 및 패딩을 적용한 후 이미지를 출력

img = cv2.imread('./1200px-Girl_and_cat.jpg')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # BGR 색상 공간을 RGB 색상 공간으로 변환
img = np.pad(img,((158,158),(0,0),(0,0)),mode='edge') # 이미지 상단과 하단에 158 픽셀씩 패딩을 추가
plt.imshow(img)

We will use pre-trained VGG-16 CNN:

In [ ]:
vgg = keras.applications.vgg16.VGG16(weights='imagenet')

- 이미지에 고양이가 있을 확률을 예측하는 함수를 정의
- 이미지넷에는 281에서 294까지 인덱싱된 여러 고양이 클래스가 있으므로 해당 클래스에 대한 확률을 더하여 전체 '고양이' 확률을 계산

In [ ]:
def predict(img):
  im = cv2.resize(img,(224,224)) # 이미지를 224x224 크기로 조정
  im = keras.applications.vgg16.preprocess_input(im) # 이미지를 VGG16 모델에 맞게 전처리
  pr = vgg.predict(np.expand_dims(im,axis=0))[0] # 이미지를 예측하고 결과를 반환
  return np.sum(pr[281:294]) # 고양이 클래스에 해당하는 값들의 합을 반환

predict(img)

이미지를 n x n 정사각형으로 나누어 확률의 히트맵을 작성

In [ ]:
def predict_map(img,n):
  dx = img.shape[0] // n # 이미지를 n x n 블록으로 나눌 때 각 블록의 크기를 계산
  res = np.zeros((n,n),dtype=np.float32) # 결과를 저장할 n x n 배열을 초기화
  for i in range(n):
    for j in range(n):
      im = img[dx*i:dx*(i+1),dx*j:dx*(j+1)] # 각 블록을 추출
      r = predict(im) # 각 블록에 대해 예측
      res[i,j] = r # 예측 값을 결과 배열에 저장
  return res

fig,ax = plt.subplots(1,2,figsize=(15,5)) # 1행 2열의 서브플롯을 생성
ax[1].imshow(img) # 원본 이미지를 오른쪽 서브플롯에 표시
ax[0].imshow(predict_map(img,10)) # 예측 결과 맵을 왼쪽 서브플롯에 표시

## Detecting Simple Objects

- 바운딩 박스의 위치를 보다 정확하게 파악하려면 **회귀 모델**을 실행하여 바운딩 박스 좌표를 예측해야 함.
- 32x32 이미지에서 검은색 직사각형을 감지하고자 하는 간단한 예부터 시작

- 다음 함수는 여러 개의 샘플 이미지를 생성

In [ ]:
# 이 코드는 다음과 같은 기능을 수행

# 1. num_imgs 개수의 이미지를 생성
# 2. 각 이미지의 크기는 img_size x img_size
# 3. 각 이미지 내에 min_object_size와 max_object_size 사이의 크기를 갖는 사각형 객체를 하나를 생성
# 4. 객체의 위치와 크기를 bboxes 배열에 저장
# 5. 생성된 이미지와 바운딩 박스를 반환

def generate_images(num_imgs, img_size=8, min_object_size = 1, max_object_size = 4):
    bboxes = np.zeros((num_imgs, 4)) # 객체의 위치와 크기를 저장할 배열을 초기화
    imgs = np.zeros((num_imgs, img_size, img_size))  # 이미지를 저장할 배열을 초기화

    for i_img in range(num_imgs):
        w, h = np.random.randint(min_object_size, max_object_size, size=2) # 객체의 너비와 높이를 랜덤하게 설정
        x = np.random.randint(0, img_size - w) # 객체의 x 좌표를 랜덤하게 설정
        y = np.random.randint(0, img_size - h) # 객체의 y 좌표를 랜덤하게 설정
        imgs[i_img, x:x+w, y:y+h] = 1.  # 이미지 내 해당 객체 영역을 1로 설정
        bboxes[i_img] = [x, y, w, h] # 객체의 위치와 크기를 저장
    return imgs, bboxes # 생성된 이미지와 바운딩 박스를 반환

# 100,000개의 이미지를 생성
imgs, bboxes = generate_images(100000)
print(f"Images shape = {imgs.shape}") # 이미지 배열의 형상을 출력
print(f"BBoxes shape = {bboxes.shape}") # 바운딩 박스 배열의 형상을 출력

네트워크의 출력을 [0;1] 범위로 만들기 위해 `bbox`를 이미지 크기로 나눔

In [ ]:
# 생성된 바운딩 박스 좌표를 이미지 크기(8)로 나누어 정규화된 좌표로 변환

bb = bboxes/8.0 # 바운딩 박스 좌표를 이미지 크기(8)로 나누어 정규화
bb[0] # 첫 번째 바운딩 박스의 정규화된 좌표를 출력

# 첫 번째 바운딩 박스의 정규화된 좌표를 출력하는 예
# 정규화된 좌표는 원래의 좌표를 이미지 크기로 나눈 값으로, 이를 통해 좌표가 0과 1 사이의 값으로 변환됨
# 예를 들어, 원래 좌표가 (x, y, w, h)라면, 정규화된 좌표는 (x/8, y/8, w/8, h/8)이 됨

dense network를 사용
실생활에서 객체의 모양이 더 복잡한 경우에는 CNN을 사용하는 것이 합리적
이 작업은 **회귀**이기 때문에 확률적 경사 하강 최적화기와 평균 제곱 오차(MSE)를 지표로 사용

In [ ]:
# Keras를 사용하여 간단한 신경망 모델을 정의하고 컴파일한 후, 모델의 요약 정보를 출력
# 이 모델은 주어진 8x8 이미지에서 4개의 값을 예측하도록 설계

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(8,8)), # 8x8 입력 이미지를 1차원 배열로 변환
    keras.layers.Dense(200, activation='relu'), # 200개의 유닛을 가진 은닉층을 추가하고 ReLU 활성화 함수를 사용
    keras.layers.Dropout(0.2), # 드롭아웃을 사용하여 과적합을 방지합니다. (20%의 유닛을 무작위로 끕니다)
    keras.layers.Dense(4) # 출력층으로 4개의 유닛을 가지며 활성화 함수는 사용하지 않음
])
model.compile('sgd','mse') # SGD(확률적 경사 하강법) 옵티마이저와 MSE(평균 제곱 오차) 손실 함수를 사용하여 모델을 컴파일
model.summary() # 모델의 요약 정보를 출력

네트워크를 훈련
약간 더 나은 성능을 위해 입력 데이터를 정규화(평균을 빼고 표준 편차로 나누는 방식)

In [ ]:
# 이미지 데이터를 정규화하고, 정규화된 데이터를 사용하여 모델을 훈련

imgs_norm = (imgs-np.mean(imgs))/np.std(imgs) # 이미지 데이터 정규화: 평균을 빼고 표준 편차로 나눔 - 데이터의 분포를 평균 0, 표준 편차 1로 만듦
model.fit(imgs_norm,bb,epochs=30) # 모델 훈련: 정규화된 이미지 데이터와 정규화된 바운딩 박스 데이터를 사용하여 모델을 30 에포크 동안 훈련

비교적 양호한 손실이 발생한 것으로 보임
이것이 mAP와 같은 보다 가시적인 지표로 어떻게 변환되는지 확인
먼저 두 바운딩 박스 사이의 IOU 메트릭을 정의

In [ ]:
# 두 바운딩 박스의 교집합 비율(IOU: Intersection Over Union)을 계산
# IOU는 두 바운딩 박스가 얼마나 겹치는지를 나타내는 지표로, 값이 1에 가까울수록 두 박스가 거의 동일한 위치와 크기를 가짐을 의미

def IOU(bbox1, bbox2):
    '''두 바운딩 박스 [x, y, w, h] 간의 교집합 비율(IOU)을 계산'''
    x1, y1, w1, h1 = bbox1[0], bbox1[1], bbox1[2], bbox1[3] # 첫 번째 바운딩 박스의 좌표와 크기
    x2, y2, w2, h2 = bbox2[0], bbox2[1], bbox2[2], bbox2[3] # 두 번째 바운딩 박스의 좌표와 크기
    w_I = min(x1 + w1, x2 + w2) - max(x1, x2)  # 교집합의 너비와 높이를 계산
    h_I = min(y1 + h1, y2 + h2) - max(y1, y2)
    if w_I <= 0 or h_I <= 0:  # 만약 교집합이 없으면 IOU는 0
        return 0.
    I = w_I * h_I # 교집합 면적을 계산
    U = w1 * h1 + w2 * h2 - I # 두 바운딩 박스의 합집합 면적을 계산
    return I / U # IOU를 반환

500개의 테스트 이미지를 생성하고 그 중 5개의 이미지를 플롯하여 얼마나 정확한지 시각화
IOU 메트릭도 인쇄

In [ ]:
# 테스트 이미지에 대한 예측 바운딩 박스를 생성하고, 실제 바운딩 박스와 비교하여 IOU를 계산한 후, 이를 시각화

import matplotlib

test_imgs, test_bboxes = generate_images(500) # # 테스트용 이미지와 바운딩 박스를 생성

# 모델을 사용하여 테스트 이미지의 바운딩 박스를 예측
# 정규화된 이미지 데이터를 사용하고, 예측된 결과를 다시 원래 크기로 변환
bb_res = model.predict((test_imgs-np.mean(imgs))/np.std(imgs))*8

plt.figure(figsize=(15,5)) # # 시각화를 위한 Figure를 생성
for i in range(5): # # 5개의 테스트 이미지에 대해 예측 결과를 출력하고 시각화
    print(f"pred={bb_res[i]},act={test_bboxes[i]}, IOU={IOU(bb_res[i],test_bboxes[i])}")
    plt.subplot(1,5,i+1)
    plt.imshow(test_imgs[i])
    plt.gca().add_patch(matplotlib.patches.Rectangle((bb_res[i,1],bb_res[i,0]),bb_res[i,3],bb_res[i,2],ec='r'))
    #plt.annotate('IOU: {:.2f}'.format(IOU(bb_res[i],test_bboxes[i])),(bb_res[i,1],bb_res[i,0]+bb_res[i,3]),color='y')


모든 사례에 대한 평균 정밀도를 계산하려면 모든 테스트 샘플을 검토하고 IoU를 계산한 다음 평균을 계산

In [ ]:
# 테스트 데이터에 대한 예측 바운딩 박스와 실제 바운딩 박스 간의 IOU 값을 계산하고, 모든 IOU 값의 평균을 계산

np.array([IOU(a,b) for a,b in zip(test_bboxes,bb_res)]).mean()